In [85]:
import numpy as np
import pandas as pd
import eikon as ek
import cufflinks as cf
import math
import DatastreamDSWS as DSWS
import scipy.optimize as sco
ek.set_app_key()
cf.set_config_file(offline=True)
ek.__version__

'1.0.1'

## Data Retrieval

In [6]:
rics = [
    'AAPL.O',  # Apple stock
    'AMZN.O',  # Amazon stock
    'SPY',  # S&P 500 ETF
    'GLD',  # Gold ETF
    'EUR=',  # EUR/USD exchange rate
]

In [14]:
data = ek.get_timeseries(rics,  # the RICs
                         fields='CLOSE',  # the required fields
                         start_date='2015-01-01',  # start date
                         end_date='2019-08-16')  # end date

print(data.shape)
data.tail()

(622, 5)


CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2019-08-12,200.48,1784.92,288.07,142.63,1.1212
2019-08-13,208.97,1824.34,292.55,141.78,1.1169
2019-08-14,202.75,1762.96,283.90,142.75,1.1138
2019-08-15,201.74,1776.12,284.65,143.70,1.1106
2019-08-16,NaN,NaN,NaN,NaN,1.1080


In [17]:
data.dropna(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 578 entries, 2017-05-01 to 2019-08-15
Data columns (total 5 columns):
AAPL.O    578 non-null float64
AMZN.O    578 non-null float64
SPY       578 non-null float64
GLD       578 non-null float64
EUR=      578 non-null float64
dtypes: float64(5)
memory usage: 27.1 KB


## Calculate single instrument statistics

Calculate the __log returns__ in vectorized fashion for all financial instruments and all days available.

In [25]:
rets = np.log(data/data.shift(1)) # ln(price_today/price_yesterdat)
rets.head()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
Date,,,,,
2017-05-01,NaN,NaN,NaN,NaN,NaN
2017-05-02,0.006325,-0.001361,0.000377,-0.000167,0.002658
2017-05-03,-0.003055,-0.006261,-0.001215,-0.014056,-0.004127
2017-05-04,-0.003610,-0.003726,0.001173,-0.010138,0.009330
2017-05-05,0.016448,-0.003612,0.003929,0.001882,0.001001


In [26]:
rets.iplot(kind = 'histogram',subplots = True)

In [29]:
# calculate annualised mean return
mean_rets = rets.mean()*252 # from daily returns
mean_rets.iplot(kind='bar')

In [36]:
std_rets = rets.std()*math.sqrt(252) #daily volatility
std_rets.iplot(kind='bar')

## Calculate Portfolio Statistics

In [64]:
#assume equal weight
weights = len(rics)*[(1/len(rics))]
print(weights)

#assume historical return is the expected return
expect_rets = rets.mean()
print(expect_rets)

#calculate portfolio returns
def portfolio_return(symbols, weights):
    return np.dot(rets[symbols].mean() * 252, weights)
portfolio_return(rics,weights)

[0.2, 0.2, 0.2, 0.2, 0.2]
CLOSE
AAPL.O    0.000554
AMZN.O    0.001088
SPY       0.000305
GLD       0.000317
EUR=      0.000033
dtype: float64


0.11573922847739435

In [48]:
#calculate covariance matrix
data.cov()

CLOSE,AAPL.O,AMZN.O,SPY,GLD,EUR=
CLOSE,,,,,
AAPL.O,481.673259,6327.933287,310.966184,-11.647726,-0.135720
AMZN.O,6327.933287,122082.574905,5216.078123,80.122478,-3.170162
SPY,310.966184,5216.078123,275.985991,18.288498,-0.071619
GLD,-11.647726,80.122478,18.288498,29.595040,0.030758
EUR=,-0.135720,-3.170162,-0.071619,0.030758,0.001372


In [51]:
#calculate portfolio variance
def portfolio_volatility(symbols, weights):
    return math.sqrt(np.dot(weights, np.dot(rets[symbols].cov() * 252, weights)))

portfolio_volatility(rics,weights)

0.12693751812649212

## Portfolio with two assets

In [61]:
fis = ['AAPL.O','AMZN.O']

#generate random portfolio composition that two elements add up to 100
w = np.random.random((500,len(fis)))
w = (w.T/w.sum(axis=1)).T # normalisation to let them add up to 100

w.sum(axis=1) #check

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [68]:
# calculate the portfolio statistics for 500 portfolios that we randomly generate weights

mvp = [(portfolio_volatility(fis, weights),
       portfolio_return(fis, weights))
         for weights in w]

mvp = pd.DataFrame(np.array(mvp),columns=['volatility','return'])
mvp

,volatility,return
0,0.251402,0.178697
1,0.251711,0.193876
2,0.253740,0.204479
3,0.262959,0.227289
4,0.256568,0.213378
5,0.259958,0.221379
6,0.252466,0.198782
7,0.251749,0.194186
8,0.257321,0.154192
9,0.260976,0.146043


In [69]:
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

## portfolio with 5 assets

In [78]:
w = np.random.random((2500,len(rics)))
w = (w.T/w.sum(axis=1)).T
w.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [80]:
mvp = [(portfolio_volatility(rics, weights),
       portfolio_return(rics, weights))
         for weights in w]

mvp = pd.DataFrame(np.array(mvp),columns=['volatility','return'])
mvp

,volatility,return
0,0.094671,0.072442
1,0.091006,0.097578
2,0.131896,0.123804
3,0.124288,0.083297
4,0.152754,0.153876
5,0.065174,0.053971
6,0.144008,0.141495
7,0.099802,0.095376
8,0.117746,0.114028
9,0.155224,0.130714


In [81]:
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

## Minimum Volatility Portfolio

In [83]:
# weight boundary for each asset
bounds = len(rics)*[(0,1)] # create a list object with 5 tuples defining lower/upper bound
bounds

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

In [84]:
# all the weight should be add up to 1
constraints = {'type':'eq','fun':lambda weights: weights.sum()-1}

In [86]:
res = sco.minimize(lambda x: portfolio_volatility(rics, x),  # function to be minized
                   len(rics) * [1 / len(rics)],  # initial guess
                   bounds=bounds,  # boundary conditions
                   constraints=constraints  # single equality constraint
                  )

In [87]:
res

     fun: 0.06071776423994474
     jac: array([0.07475597, 0.0882541 , 0.06070363, 0.06059043, 0.06074834])
 message: 'Optimization terminated successfully.'
    nfev: 70
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.00000000e+00, 5.55203470e-18, 1.83291448e-01, 1.41733486e-01,
       6.74975066e-01])

In [88]:
res['fun'] #minimum volatility

0.06071776423994474

In [89]:
for r in zip(rics, res['x']):
    print('%7s | %7.3f' % (r[0], r[1])) # optimal portfolio composition

 AAPL.O |   0.000
 AMZN.O |   0.000
    SPY |   0.183
    GLD |   0.142
   EUR= |   0.675


This is the tutorial from https://github.com/yhilpisch/eikondataapi/blob/master/notebooks/06_tpq_eikon_portfolio_selection.ipynb